# Dealing With Missing Data

In [363]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource,HoverTool,ColorBar
from bokeh.plotting import figure, show, output_file

output_notebook()

data = pd.read_csv('Wholesale customers data - Missing Values.csv')
len(data)

Loading BokehJS ...

440

## Finding records containing missing values

In [364]:
data.isnull().sum()

Channel             0
Region              0
Fresh               0
Milk                1
Grocery             2
Frozen              0
Detergents_Paper    2
Delicassen          1
dtype: int64

In [365]:
data[data['Milk'].isnull()]

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,NaN,8887.0,402,1382.0,2794.0


In [366]:
data[data['Delicassen'].isnull()]

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
274,1,3,894,1703.0,1841.0,744,759.0,NaN


In [367]:
data[data['Grocery'].isnull()]

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
75,1,3,20398,1137.0,NaN,4407,3.0,975.0
172,1,3,955,5479.0,NaN,333,2840.0,707.0


In [368]:
data[data['Detergents_Paper'].isnull()]

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
225,1,1,12680,3243.0,4157.0,660,NaN,786.0
370,2,3,39679,3944.0,4955.0,1364,NaN,2235.0


### So, we've 6 missing values in the dataset - One under Milk, two under Grocery, two under Detergents_Paper and one under Delicassen.

## Visualizing dataset

In [369]:
data = pd.read_csv('Wholesale customers data - Missing Values.csv')

df = data
df['ID'] = df['Channel'].astype(str) + df['Region'].astype(str)
df = df.drop('Channel',axis=1)
df = df.drop('Region',axis=1)

df = df.set_index('ID')
pivoted_df = pd.DataFrame(df.stack(), columns=['Value']).reset_index()
pivoted_df = pivoted_df.rename(columns={'level_1': 'Product'})
pivoted_df = pivoted_df[pivoted_df['Product']!='Channel']
pivoted_df = pivoted_df[pivoted_df['Product']!='Region']

colormap = {'11': 'red', '12': 'blue', '13': 'green', '21': 'yellow', '22': 'magenta', '23': 'orange'}
colors = [colormap[x] for x in pivoted_df['ID']]

prods = pivoted_df['Product']
ids = pivoted_df['ID']
vals = pivoted_df['Value']

df = pd.DataFrame(np.column_stack([prods, ids, vals]), columns=['prods', 'ids', 'vals'])
source = ColumnDataSource(df)

p = figure(title = "Wholesale Customers Data", x_range = ['Fresh','Milk','Grocery','Frozen','Detergents_Paper','Delicassen'], y_range = (0,113000), tools="hover,lasso_select,pan,wheel_zoom,box_zoom,reset,save")
p.xaxis.axis_label = 'Product'
p.yaxis.axis_label = 'Expenditure'

p.asterisk(x='prods', y='vals', color=colors, source=source, fill_alpha=0.2, size=12)
p.select_one(HoverTool).tooltips = [('Product', '@prods'),('Store ID', '@ids'),('Expenditure', '@vals')]
show(p)

#The depreciation warning has been ignored since I used only column names for x and y
#corresponding to columns in the data source, just as recommended. 
#Besides, since only column names, and not iterable values, are used,
#there's no chance of data in the columndatasource object getting modified.

D:\Anaconda\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
D:\Anaconda\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


The products form the X axis, the expenditure forms the Y axis and the different Channel-Region (ID)s are represented in different colors.
Please use box or wheel zoom to zero in on the densely packed clusters of data points, and lasso tool to place emphasis on certain data points.

## Determining actual values of these missing values:

In [370]:
act_data = pd.read_csv('Wholesale customers data - No Missing Values.csv')
print(act_data.iloc[180]['Milk'], act_data.iloc[274]['Delicassen'], act_data.iloc[75]['Grocery'], act_data.iloc[172]['Grocery'], act_data.iloc[225]['Detergents_Paper'], act_data.iloc[370]['Detergents_Paper'])

6036 1153 3 6536 761 523


## Data Cleaning Approach 1: Ignoring tuples with missing values

In [371]:
data_1 = data
data_1 = data_1.dropna()
len(data_1)

434

In [372]:
data_1.isnull().sum()

Channel             0
Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
ID                  0
dtype: int64

All records with even one missing value have been removed. The plot for this will simply be the same as the one above since the visualization plots products vs expenditure with a color code for different Channel-Region ID.

Note: In the visualizations below, squares and circles are used for plotting imputed and actual values respectively of the six missing values from the dataset. The actual values were referred from the original dataset contatining no missing values.

## Data Cleaning Approach 2: Using a global constant

In [373]:
data_3 = data
data_3.isnull().sum()

Channel             0
Region              0
Fresh               0
Milk                1
Grocery             2
Frozen              0
Detergents_Paper    2
Delicassen          1
ID                  0
dtype: int64

In [374]:
data_3 = data_3.fillna(-1)
data_3.isnull().sum()

Channel             0
Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
ID                  0
dtype: int64

In [375]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource,HoverTool


ac_y = ['Milk','Grocery_1', 'Grocery_2', 'Detergents_Paper 1', 'Detergents_Paper 2','Delicassen']
#Note: ac_x denotes the list of actual values referenced from the dataset containing no missing values.
ac_x = [6036, 3, 6536, 761, 523, 1153]
imp_x = [-1]*6

df = pd.DataFrame(np.column_stack([ac_y, ac_x, imp_x]), columns=['ac_y', 'ac_x', 'imp_x'])
source = ColumnDataSource(df)

p = figure(y_range=ac_y, tools="hover,lasso_select,pan,wheel_zoom,box_zoom,reset,save")
p.xaxis.axis_label = 'Expenditure'
p.yaxis.axis_label = 'Product'

p.circle(x='ac_x', y='ac_y', fill_color="black", legend = 'Actual', source=source, size=8)
p.square(x='imp_x', y='ac_y', fill_color="red", legend = 'Imputed', source=source, size=8)
p.select_one(HoverTool).tooltips = [('Product', '@ac_y'),('Actual annual spending on the product', '@ac_x'),('Imputed annual spending on the product', '@imp_x')]
show(p)

Needless to say, this is easily the most unfavorable way to make up for missing values.

## Data Cleaning Approach 3: Using the attribute mean

In [376]:
data_4 = data
data_4.isnull().sum()

Channel             0
Region              0
Fresh               0
Milk                1
Grocery             2
Frozen              0
Detergents_Paper    2
Delicassen          1
ID                  0
dtype: int64

In [377]:
milk_mean = np.mean(data_4['Milk'])
grocery_mean = np.mean(data_4['Grocery'])
detergent_mean = np.mean(data_4['Detergents_Paper'])
delicassen_mean = np.mean(data_4['Delicassen'])
print(milk_mean,grocery_mean,detergent_mean,delicassen_mean)

5795.719817767654 7972.655251141552 2891.719178082192 1525.7175398633258


In [378]:
data_4['Milk'] = data_4['Milk'].fillna(milk_mean)
data_4['Grocery'] = data_4['Grocery'].fillna(grocery_mean)
data_4['Detergents_Paper'] = data_4['Detergents_Paper'].fillna(detergent_mean)
data_4['Delicassen'] = data_4['Delicassen'].fillna(delicassen_mean)

data_4.isnull().sum()

Channel             0
Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
ID                  0
dtype: int64

In [379]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource,HoverTool


ac_y = ['Milk','Grocery_1', 'Grocery_2', 'Detergents_Paper 1', 'Detergents_Paper 2','Delicassen']
#Note: ac_x denotes the list of actual values referenced from the dataset containing no missing values.
ac_x = [6036, 3, 6536, 761, 523, 1153]
imp_x = [milk_mean, grocery_mean, grocery_mean, detergent_mean, detergent_mean, delicassen_mean]
imp_x = [int(imp) for imp in imp_x]

df = pd.DataFrame(np.column_stack([ac_y, ac_x, imp_x]), columns=['ac_y', 'ac_x', 'imp_x'])
source = ColumnDataSource(df)

p = figure(y_range=ac_y, tools="hover,lasso_select,pan,wheel_zoom,box_zoom,reset,save")
p.xaxis.axis_label = 'Expenditure'
p.yaxis.axis_label = 'Product'

p.circle(x='ac_x', y='ac_y', fill_color="black", legend = 'Actual', source=source, size=8)
p.square(x='imp_x', y='ac_y', fill_color="red", legend = 'Imputed', source=source, size=8)
p.select_one(HoverTool).tooltips = [('Product', '@ac_y'),('Actual annual spending on the product', '@ac_x'),('Imputed annual spending on the product', '@imp_x')]
show(p)

As can be seen, filling up missing values using attribute mean is slightly better. For example, the imputed milk and delicassen values aren't relatively too far off from the actual values.

## Data Cleaning Approach 4: Filling in the missing values manually

In [380]:
data_2 = pd.read_csv('Wholesale customers data - Missing Values.csv')
data_2.isnull().sum()

Channel             0
Region              0
Fresh               0
Milk                1
Grocery             2
Frozen              0
Detergents_Paper    2
Delicassen          1
dtype: int64

In [381]:
data_2.loc[180,'Milk'] = 5000
data_2.loc[274,'Delicassen'] = 1200
data_2.loc[75,'Grocery'] = 20000
data_2.loc[172,'Grocery'] = 30000
data_2.loc[225,'Detergents_Paper'] = 8000
data_2.loc[370,'Detergents_Paper'] = 10000

In [382]:
data_2.isnull().sum()

Channel             0
Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
dtype: int64

In [383]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource,HoverTool


ac_y = ['Milk','Grocery_1', 'Grocery_2', 'Detergents_Paper 1', 'Detergents_Paper 2','Delicassen']
#Note: ac_x denotes the list of actual values referenced from the dataset containing no missing values.
ac_x = [6036, 3, 6536, 761, 523, 1153]
imp_x = [5000, 20000, 30000, 8000, 10000, 1200]

df = pd.DataFrame(np.column_stack([ac_y, ac_x, imp_x]), columns=['ac_y', 'ac_x', 'imp_x'])
source = ColumnDataSource(df)
p = figure(y_range=ac_y, tools="hover,lasso_select,pan,wheel_zoom,box_zoom,reset,save")
p.xaxis.axis_label = 'Expenditure'
p.yaxis.axis_label = 'Product'

p.circle(x='ac_x', y='ac_y', fill_color="black", legend = 'Actual', source=source, size=8)
p.square(x='imp_x', y='ac_y', fill_color="red", legend = 'Imputed', source=source, size=8)
p.select_one(HoverTool).tooltips = [('Product', '@ac_y'),('Actual annual spending on the product', '@ac_x'),('Imputed annual spending on the product', '@imp_x')]
show(p)

One requires knowledge of the domain to make informed decisions on values that can fill up the missing cells. So the effectiveness of this approach depends entirely on that. Notice how one of the manually entered values is fairly accurate while the rest are generally very deviant.

## Data Cleaning Approach 5: Filling in most probable values (Using KNN ML Model)

In [384]:
data_5 = pd.read_csv('Wholesale customers data - Missing Values.csv')

train_df = data_5
train_df = train_df.dropna()

null_cols = [180, 274, 75, 172, 225, 370]
test_df = data_5
test_df = test_df.iloc[null_cols]
test_df

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,NaN,8887.0,402,1382.0,2794.0
274,1,3,894,1703.0,1841.0,744,759.0,NaN
75,1,3,20398,1137.0,NaN,4407,3.0,975.0
172,1,3,955,5479.0,NaN,333,2840.0,707.0
225,1,1,12680,3243.0,4157.0,660,NaN,786.0
370,2,3,39679,3944.0,4955.0,1364,NaN,2235.0


In [385]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score, KFold

#Using milk as target column and the rest as feature columns
features = list(train_df.columns.values)
features.remove('Milk')
features.remove('Channel')
features.remove('Region')

knn = KNeighborsRegressor(n_neighbors=7)
knn.fit(train_df[features],train_df['Milk'])

kf = KFold(15, shuffle=True, random_state=1)
mses = cross_val_score(knn, train_df[features],train_df['Milk'], scoring="neg_mean_squared_error", cv=kf)
rmses = [np.sqrt(np.absolute(x)) for x in mses]
avg_rmse = round(np.mean(rmses),2)
print("By optimizing the K (Number of neighbors) and k (No of cross validation folds), the best average RMSE is found to be: ", avg_rmse)

milk_predicted = knn.predict(test_df.loc[180][features].values.reshape(1,-1))
milk_predicted = milk_predicted[0]
test_df = test_df.set_value(180,'Milk',milk_predicted)
test_df

By optimizing the K (Number of neighbors) and k (No of cross validation folds), the best average RMSE is found to be:  4653.36


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,6859.428571,8887.0,402,1382.0,2794.0
274,1,3,894,1703.000000,1841.0,744,759.0,NaN
75,1,3,20398,1137.000000,NaN,4407,3.0,975.0
172,1,3,955,5479.000000,NaN,333,2840.0,707.0
225,1,1,12680,3243.000000,4157.0,660,NaN,786.0
370,2,3,39679,3944.000000,4955.0,1364,NaN,2235.0


In [386]:
#Using grocery as target column and the rest as feature columns
features = list(train_df.columns.values)
features.remove('Grocery')
features.remove('Channel')
features.remove('Region')

knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(train_df[features],train_df['Grocery'])

kf = KFold(20, shuffle=True, random_state=1)
mses = cross_val_score(knn, train_df[features],train_df['Grocery'], scoring="neg_mean_squared_error", cv=kf)
rmses = [np.sqrt(np.absolute(x)) for x in mses]
avg_rmse = round(np.mean(rmses),2)
print("By optimizing the K (Number of neighbors) and k (No of cross validation folds), the best average RMSE is found to be: ", avg_rmse)

grocery_predicted_1 = knn.predict(test_df.loc[75][features].values.reshape(1,-1))
grocery_predicted_2 = knn.predict(test_df.loc[172][features].values.reshape(1,-1))
grocery_predicted_1 = grocery_predicted_1[0]
grocery_predicted_2 = grocery_predicted_2[0]
test_df.set_value(75,'Grocery',grocery_predicted_1)
test_df.set_value(172,'Grocery',grocery_predicted_2)
test_df

By optimizing the K (Number of neighbors) and k (No of cross validation folds), the best average RMSE is found to be:  4315.64


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,6859.428571,8887.0,402,1382.0,2794.0
274,1,3,894,1703.000000,1841.0,744,759.0,NaN
75,1,3,20398,1137.000000,2767.4,4407,3.0,975.0
172,1,3,955,5479.000000,7615.2,333,2840.0,707.0
225,1,1,12680,3243.000000,4157.0,660,NaN,786.0
370,2,3,39679,3944.000000,4955.0,1364,NaN,2235.0


In [387]:
#Using detergent_paper as target column and the rest as feature columns
features = list(train_df.columns.values)
features.remove('Detergents_Paper')
features.remove('Channel')
features.remove('Region')

knn = KNeighborsRegressor(n_neighbors=7)
knn.fit(train_df[features],train_df['Detergents_Paper'])

kf = KFold(20, shuffle=True, random_state=1)
mses = cross_val_score(knn, train_df[features],train_df['Detergents_Paper'], scoring="neg_mean_squared_error", cv=kf)
rmses = [np.sqrt(np.absolute(x)) for x in mses]
avg_rmse = round(np.mean(rmses),2)
print("By optimizing the K (Number of neighbors) and k (No of cross validation folds), the best average RMSE is found to be: ", avg_rmse)

Detergents_Paper_predicted_1 = knn.predict(test_df.loc[225][features].values.reshape(1,-1))
Detergents_Paper_predicted_2 = knn.predict(test_df.loc[370][features].values.reshape(1,-1))
Detergents_Paper_predicted_1 = Detergents_Paper_predicted_1[0]
Detergents_Paper_predicted_2 = Detergents_Paper_predicted_2[0]
test_df.set_value(225,'Detergents_Paper',Detergents_Paper_predicted_1)
test_df.set_value(370,'Detergents_Paper',Detergents_Paper_predicted_2)
test_df

By optimizing the K (Number of neighbors) and k (No of cross validation folds), the best average RMSE is found to be:  1938.33


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,6859.428571,8887.0,402,1382.000000,2794.0
274,1,3,894,1703.000000,1841.0,744,759.000000,NaN
75,1,3,20398,1137.000000,2767.4,4407,3.000000,975.0
172,1,3,955,5479.000000,7615.2,333,2840.000000,707.0
225,1,1,12680,3243.000000,4157.0,660,1281.000000,786.0
370,2,3,39679,3944.000000,4955.0,1364,705.571429,2235.0


In [388]:
#Using Delicassen as target column and the rest as feature columns
features = list(train_df.columns.values)
features.remove('Delicassen')
features.remove('Channel')
features.remove('Region')

knn = KNeighborsRegressor(n_neighbors=14)
knn.fit(train_df[features],train_df['Delicassen'])

kf = KFold(23, shuffle=True, random_state=1)
mses = cross_val_score(knn, train_df[features],train_df['Delicassen'], scoring="neg_mean_squared_error", cv=kf)
rmses = [np.sqrt(np.absolute(x)) for x in mses]
avg_rmse = round(np.mean(rmses),2)
print("By optimizing the K (Number of neighbors) and k (No of cross validation folds), the best average RMSE is found to be: ", avg_rmse)

Delicassen_predicted = knn.predict(test_df.loc[274][features].values.reshape(1,-1))
Delicassen_predicted = Delicassen_predicted[0]
test_df.set_value(274,'Delicassen',Delicassen_predicted)
test_df

By optimizing the K (Number of neighbors) and k (No of cross validation folds), the best average RMSE is found to be:  1836.99


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,6859.428571,8887.0,402,1382.000000,2794.0
274,1,3,894,1703.000000,1841.0,744,759.000000,572.5
75,1,3,20398,1137.000000,2767.4,4407,3.000000,975.0
172,1,3,955,5479.000000,7615.2,333,2840.000000,707.0
225,1,1,12680,3243.000000,4157.0,660,1281.000000,786.0
370,2,3,39679,3944.000000,4955.0,1364,705.571429,2235.0


In [389]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource,HoverTool


ac_y = ['Milk','Grocery_1', 'Grocery_2', 'Detergents_Paper 1', 'Detergents_Paper 2','Delicassen']
ac_x = [6036, 3, 6536, 761, 523, 1153]
imp_x = [milk_predicted, grocery_predicted_1, grocery_predicted_2, Detergents_Paper_predicted_1, Detergents_Paper_predicted_2, Delicassen_predicted]
imp_x = [int(imp) for imp in imp_x]

df = pd.DataFrame(np.column_stack([ac_y, ac_x, imp_x]), columns=['ac_y', 'ac_x', 'imp_x'])
source = ColumnDataSource(df)
p = figure(y_range=ac_y, tools="hover,lasso_select,pan,wheel_zoom,box_zoom,reset,save")
p.xaxis.axis_label = 'Expenditure'
p.yaxis.axis_label = 'Product'

p.circle(x='ac_x', y='ac_y', fill_color="black", legend = 'Actual', source=source, size=8)
p.square(x='imp_x', y='ac_y', fill_color="red", legend = 'Imputed', source=source, size=8)
p.select_one(HoverTool).tooltips = [('Product', '@ac_y'),('Actual annual spending on the product', '@ac_x'),('Imputed annual spending on the product', '@imp_x')]
show(p)

As seen from the above plot, the KNN model does a fair job of determining values for imputation. However, KKN isn't quite a mathematical model. Let's see if we can improve the imputation any further using the linear regression model.

## Data Cleaning Approach 6: Filling in most probable values (Using Linear Regression model)

In [390]:
data_6 = pd.read_csv('Wholesale customers data - Missing Values.csv')

train_df = data_6
train_df = train_df.dropna()

null_cols = [180, 274, 75, 172, 225, 370]
test_df = data_6
test_df = test_df.iloc[null_cols]
test_df

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,NaN,8887.0,402,1382.0,2794.0
274,1,3,894,1703.0,1841.0,744,759.0,NaN
75,1,3,20398,1137.0,NaN,4407,3.0,975.0
172,1,3,955,5479.0,NaN,333,2840.0,707.0
225,1,1,12680,3243.0,4157.0,660,NaN,786.0
370,2,3,39679,3944.0,4955.0,1364,NaN,2235.0


In [391]:
from sklearn.linear_model import LinearRegression

#Using milk as target column and the rest as feature columns
features = list(train_df.columns.values)
features.remove('Milk')
features.remove('Channel')
features.remove('Region')

lr = LinearRegression()
lr.fit(train_df[features],train_df['Milk'])

kf = KFold(10, shuffle=True, random_state=1)
mses = cross_val_score(lr, train_df[features],train_df['Milk'], scoring="neg_mean_squared_error", cv=kf)
rmses = [np.sqrt(np.absolute(x)) for x in mses]
avg_rmse = round(np.mean(rmses),2)
print("The average RMSE is found to be: ", avg_rmse)

milk_predicted_lr = lr.predict(test_df.loc[180][features].values.reshape(1,-1))
milk_predicted_lr = milk_predicted_lr[0]
test_df = test_df.set_value(180,'Milk',milk_predicted_lr)
test_df

The average RMSE is found to be:  4440.54


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,6354.883201,8887.0,402,1382.0,2794.0
274,1,3,894,1703.000000,1841.0,744,759.0,NaN
75,1,3,20398,1137.000000,NaN,4407,3.0,975.0
172,1,3,955,5479.000000,NaN,333,2840.0,707.0
225,1,1,12680,3243.000000,4157.0,660,NaN,786.0
370,2,3,39679,3944.000000,4955.0,1364,NaN,2235.0


In [392]:
#Using grocery as target column and the rest as feature columns
features = list(train_df.columns.values)
features.remove('Grocery')
features.remove('Channel')
features.remove('Region')

lr = LinearRegression()
lr.fit(train_df[features],train_df['Grocery'])

kf = KFold(20, shuffle=True, random_state=1)
mses = cross_val_score(lr, train_df[features],train_df['Grocery'], scoring="neg_mean_squared_error", cv=kf)
rmses = [np.sqrt(np.absolute(x)) for x in mses]
avg_rmse = round(np.mean(rmses),2)
print("The average RMSE is found to be: ", avg_rmse)

grocery_predicted_1_lr = lr.predict(test_df.loc[75][features].values.reshape(1,-1))
grocery_predicted_2_lr = lr.predict(test_df.loc[172][features].values.reshape(1,-1))
grocery_predicted_1_lr = grocery_predicted_1_lr[0]
grocery_predicted_2_lr = grocery_predicted_2_lr[0]
test_df.set_value(75,'Grocery',grocery_predicted_1_lr)
test_df.set_value(172,'Grocery',grocery_predicted_2_lr)
test_df

The average RMSE is found to be:  3256.59


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,6354.883201,8887.000000,402,1382.0,2794.0
274,1,3,894,1703.000000,1841.000000,744,759.0,NaN
75,1,3,20398,1137.000000,2446.054756,4407,3.0,975.0
172,1,3,955,5479.000000,7265.688676,333,2840.0,707.0
225,1,1,12680,3243.000000,4157.000000,660,NaN,786.0
370,2,3,39679,3944.000000,4955.000000,1364,NaN,2235.0


In [393]:
#Using detergent_paper as target column and the rest as feature columns
features = list(train_df.columns.values)
features.remove('Detergents_Paper')
features.remove('Channel')
features.remove('Region')

lr = LinearRegression()
lr.fit(train_df[features],train_df['Detergents_Paper'])

kf = KFold(20, shuffle=True, random_state=1)
mses = cross_val_score(lr, train_df[features],train_df['Detergents_Paper'], scoring="neg_mean_squared_error", cv=kf)
rmses = [np.sqrt(np.absolute(x)) for x in mses]
avg_rmse = round(np.mean(rmses),2)
print("The average RMSE is found to be: ", avg_rmse)

Detergents_Paper_predicted_1_lr = lr.predict(test_df.loc[225][features].values.reshape(1,-1))
Detergents_Paper_predicted_2_lr = lr.predict(test_df.loc[370][features].values.reshape(1,-1))
Detergents_Paper_predicted_1_lr = Detergents_Paper_predicted_1_lr[0]
Detergents_Paper_predicted_2_lr = Detergents_Paper_predicted_2_lr[0]
test_df.set_value(225,'Detergents_Paper',Detergents_Paper_predicted_1_lr)
test_df.set_value(370,'Detergents_Paper',Detergents_Paper_predicted_2_lr)
test_df

The average RMSE is found to be:  1651.51


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,6354.883201,8887.000000,402,1382.000000,2794.0
274,1,3,894,1703.000000,1841.000000,744,759.000000,NaN
75,1,3,20398,1137.000000,2446.054756,4407,3.000000,975.0
172,1,3,955,5479.000000,7265.688676,333,2840.000000,707.0
225,1,1,12680,3243.000000,4157.000000,660,1288.760861,786.0
370,2,3,39679,3944.000000,4955.000000,1364,807.626880,2235.0


In [394]:
#Using Delicassen as target column and the rest as feature columns
features = list(train_df.columns.values)
features.remove('Delicassen')
features.remove('Channel')
features.remove('Region')

lr = LinearRegression()
lr.fit(train_df[features],train_df['Delicassen'])

kf = KFold(23, shuffle=True, random_state=1)
mses = cross_val_score(lr, train_df[features],train_df['Delicassen'], scoring="neg_mean_squared_error", cv=kf)
rmses = [np.sqrt(np.absolute(x)) for x in mses]
avg_rmse = round(np.mean(rmses),2)
print("The average RMSE is found to be: ", avg_rmse)

Delicassen_predicted_lr = lr.predict(test_df.loc[274][features].values.reshape(1,-1))
Delicassen_predicted_lr = Delicassen_predicted_lr[0]
test_df.set_value(274,'Delicassen',Delicassen_predicted_lr)
test_df

The average RMSE is found to be:  2094.46


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,6354.883201,8887.000000,402,1382.000000,2794.000000
274,1,3,894,1703.000000,1841.000000,744,759.000000,282.665036
75,1,3,20398,1137.000000,2446.054756,4407,3.000000,975.000000
172,1,3,955,5479.000000,7265.688676,333,2840.000000,707.000000
225,1,1,12680,3243.000000,4157.000000,660,1288.760861,786.000000
370,2,3,39679,3944.000000,4955.000000,1364,807.626880,2235.000000


In [395]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource,HoverTool


ac_y = ['Milk','Grocery_1', 'Grocery_2', 'Detergents_Paper 1', 'Detergents_Paper 2','Delicassen']
ac_x = [6036, 3, 6536, 761, 523, 1153]
imp_x = [milk_predicted_lr, grocery_predicted_1_lr, grocery_predicted_2_lr, Detergents_Paper_predicted_1_lr, Detergents_Paper_predicted_2_lr, Delicassen_predicted_lr]
imp_x = [int(imp) for imp in imp_x]

df = pd.DataFrame(np.column_stack([ac_y, ac_x, imp_x]), columns=['ac_y', 'ac_x', 'imp_x'])
source = ColumnDataSource(df)
p = figure(y_range=ac_y, tools="hover,lasso_select,pan,wheel_zoom,box_zoom,reset,save")
p.xaxis.axis_label = 'Expenditure'
p.yaxis.axis_label = 'Product'

p.circle(x='ac_x', y='ac_y', fill_color="black", legend = 'Actual', source=source, size=8)
p.square(x='imp_x', y='ac_y', fill_color="red", legend = 'Imputed', source=source, size=8)
p.select_one(HoverTool).tooltips = [('Product', '@ac_y'),('Actual annual spending on the product', '@ac_x'),('Imputed annual spending on the product', '@imp_x')]
show(p)

The Linear Regression model proves to be marginally better than the K-NN model by determining imputed values that are closer to the actual values.